In [1]:
import os
import copy
from pathlib import Path
import json
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import itertools

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential

import hls4ml
from hls4ml.converters.keras_to_hls import parse_default_keras_layer
from hls4ml.model.attributes import ConfigurableAttribute, TypeAttribute
from hls4ml.model.types import FixedPrecisionType, RoundingMode, SaturationMode
from hls4ml.model.attributes import Attribute

import h5py

import qkeras as qk
from qkeras.estimate import print_qstats
from qkeras.utils import model_quantize
from qkeras.utils import quantized_model_dump
from qkeras import QActivation, QDense, QConv2DBatchnorm

# Source the Vivado path
os.environ['PATH'] = os.environ['XILINX_VIVADO'] + '/bin:' + os.environ['PATH']

np.random.seed(0)
tf.random.set_seed(0)


2025-07-01 17:29:13.970159: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-01 17:29:14.058271: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-01 17:29:14.061136: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-07-01 17:29:14.061150: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

/home/aelabd/RHEED/hls4ml/hls4ml/converters/__init__.py:29: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!", stacklevel=1)


Helper functions

In [2]:
def ap_fixed_bin_to_float(bin_str: str, X: int, Y: int) -> np.float32:
    """
    Converts a binary string in two's complement ap_fixed<X,Y> format to a float.
    
    Parameters:
        bin_str (str): Binary string of length X.
        X (int): Total number of bits.
        Y (int): Number of integer bits (including sign bit).
        
    Returns:
        np.float32: Floating point value.
    """
    if len(bin_str) != X:
        raise ValueError(f"Binary string length ({len(bin_str)}) must match X ({X})")
    
    # Interpret binary string as signed integer
    int_val = int(bin_str, 2)
    if bin_str[0] == '1':  # Negative number (two's complement)
        int_val -= (1 << X)
    
    # Scale back by number of fractional bits
    frac_bits = X - Y
    value = int_val / (2 ** frac_bits)
    return np.float32(value)

print(ap_fixed_bin_to_float('00110100', 8, 4))  # 3.25
print(ap_fixed_bin_to_float('11100000', 8, 4))  # -4

3.25
-2.0


# Load QKeras output (benchmark)

In [ ]:
IN_ROWS = 100
IN_COLS = 160
OUT_ROWS = 48
OUT_COLS = 48
NUM_CROPS = 1
data_dir = f"tb_data_Mono8/{IN_ROWS}x{IN_COLS}_to_{OUT_ROWS}x{OUT_COLS}x{NUM_CROPS}"

CROP_X0 = [0, 13, 112]
CROP_Y0 = [0, 1, 52]

TOTAL_BITS = 22
INT_BITS = 11

qkeras_out = np.zeros((6, 5))
i = 0
for y0 in CROP_Y0:
    for x0 in CROP_X0:
        if y0==0 and x0==13: continue
        fpath = os.path.join(data_dir, f"Y1_{y0}/X1_{x0}/QKeras_mg1_pred_ap_fixed_22_11.txt")
        with open(fpath, "r") as f:
            for j, line in enumerate(f.readlines()):
                line_float = ap_fixed_bin_to_float(line.strip(), TOTAL_BITS, INT_BITS)
                qkeras_out[i, j] = line_float
            i += 1

print(qkeras_out)

[[ 0.31640625  0.45166016  0.4921875   0.37255859 -0.08935547]
 [ 0.32226562  0.44970703  0.47998047  0.3515625  -0.10498047]
 [ 0.31640625  0.45166016  0.4921875   0.37255859 -0.08935547]
 [ 0.31103516  0.34960938  0.41845703  0.25244141 -0.13134766]
 [ 0.3046875   0.35742188  0.40332031  0.24804688 -0.15966797]
 [ 0.31103516  0.34960938  0.41845703  0.25244141 -0.13134766]]


# Load RTL CNN output

In [ ]:
rtl_cnn_out = np.zeros((6,5))
i=0
for y0 in CROP_Y0:
    for x0 in CROP_X0:
        if y0==0 and x0==13: continue
        fpath = os.path.join(data_dir, f"Y1_{y0}/X1_{x0}/CNN_mg1_out_benchmark_ap_fixed_22_11.txt")
        with open(fpath, "r") as f:
            for j, line in enumerate(f.readlines()):
                line_float = ap_fixed_bin_to_float(line.strip(), TOTAL_BITS, INT_BITS)
                rtl_cnn_out[i, j] = line_float
            i += 1

print(rtl_cnn_out)

[[ 0.32714844  0.45507812  0.49121094  0.34960938 -0.09716797]
 [ 0.31640625  0.45263672  0.50048828  0.3671875  -0.08496094]
 [ 0.32714844  0.45507812  0.49121094  0.34960938 -0.09716797]
 [ 0.30615234  0.34472656  0.42431641  0.25537109 -0.12207031]
 [ 0.29980469  0.35253906  0.40917969  0.25097656 -0.15039062]
 [ 0.30615234  0.34472656  0.42431641  0.25537109 -0.12207031]]


# Load RTL full pipeline output

In [13]:
full_pipeline_out = np.zeros((6,5))
i=0
for y0 in CROP_Y0:
    for x0 in CROP_X0:
        if y0==0 and x0==13: continue
        fpath = os.path.join(data_dir, f"Y1_{y0}/X1_{x0}/full_pipeline_mg1_out_ap_fixed_22_11.txt")
        with open(fpath, "r") as f:
            for j,line in enumerate(f.readlines()):
                line_float = ap_fixed_bin_to_float(line.strip(), 22, 11)
                full_pipeline_out[i, j] = line_float
            i += 1

print(full_pipeline_out)

[[ 0.31640625  0.45166016  0.4921875   0.37255859 -0.08935547]
 [ 0.32226562  0.44970703  0.47998047  0.3515625  -0.10498047]
 [ 0.31640625  0.45166016  0.4921875   0.37255859 -0.08935547]
 [ 0.31103516  0.34960938  0.41845703  0.25244141 -0.13134766]
 [ 0.3046875   0.35742188  0.40332031  0.24804688 -0.15966797]
 [ 0.31103516  0.34960938  0.41845703  0.25244141 -0.13134766]]


# Compare and contrast

In [14]:
# QKeras CNN - RTL CNN 
print(qkeras_out - rtl_cnn_out)

[[-0.01074219 -0.00341797  0.00097656  0.02294922  0.0078125 ]
 [ 0.00585938 -0.00292969 -0.02050781 -0.015625   -0.02001953]
 [-0.01074219 -0.00341797  0.00097656  0.02294922  0.0078125 ]
 [ 0.00488281  0.00488281 -0.00585938 -0.00292969 -0.00927734]
 [ 0.00488281  0.00488281 -0.00585938 -0.00292969 -0.00927734]
 [ 0.00488281  0.00488281 -0.00585938 -0.00292969 -0.00927734]]


In [15]:
print(qkeras_out - full_pipeline_out)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
